## Overview
In the previous mission, we learned how to read JSON into a Spark DataFrame, as well as some basic techniques for interacting with DataFrames. In this mission, we'll learn how to use Spark's SQL interface to query and interact with the data. We'll continue to work with the 2010 U.S. Census data set in this mission. Later on, we'll add other files to demonstrate how to take advantage of SQL to work with multiple data sets.

## Register the DataFrame as a Table
Before we can write and run SQL queries, we need to tell Spark to treat the DataFrame as a SQL table. Spark internally maintains a virtual database within the SQLContext object. This object, which we enter as __sqlCtx__, has methods for registering temporary tables.

To register a DataFrame as a table, call the __registerTempTable()__ [method](https://spark.apache.org/docs/1.5.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.registerTempTable) on that DataFrame object. This method requires one string parameter, name, that we use to set the table __name__ for reference in our SQL queries.

#### Instructions
* Use the __registerTempTable()__ [method](https://spark.apache.org/docs/1.5.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.registerTempTable) method to register the DataFrame __df__ as a table named __census2010__.
* Then, run the SQLContext method __tableNames__ to return the list of tables.
* Assign the resulting list to __tables__, and use the __print__ function to display it.

In [1]:
# Find path to PySpark
import findspark
findspark.init()

# Import PySpark & initalize SparkContext object
import pyspark
sc = pyspark.SparkContext()

# Import SQLContext
from pyspark.sql import SQLContext

# Pass in the SparkContext object `sc`
sqlCtx = SQLContext(sc)

# Read JSON data into a DataFrame object `df`
df = sqlCtx.read.json("census_2010.json")
df.registerTempTable('census2010')

In [3]:
tables = sqlCtx.tableNames()
print(tables)

[u'census2010']


## Querying
Now that we've registered the table within __sqlCtx__, we can start writing and running SQL queries. With Spark SQL, we represent our query as a string and pass it into the __sql()__ [method](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SQLContext.sql) within the SQLContext object. The __sql()__ method requires a single parameter, the query string. Spark will return the query results as a spark DataFrame object. This means you'll have to use __show()__ to display the results, due to lazy loading.

While SQLite requires that queries end with a semi-colon, Spark SQL will actually throw an error if you include it. Other than this difference in syntax, Spark's flavor of SQL is identical to SQLite, and all the queries you've written for the SQL course will work here as well.

#### Instructions
* Write a SQL query that returns the __age__ column from the table __census2010__ and the __show()__ method to display the first 20 results.

In [16]:
print(sqlCtx.sql("SELECT age FROM census2010 LIMIT 20").show())



+---+
|age|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+

None


## Filtering
In the previous mission, we used DataFrame methods to find all of the rows where __age__ was greater than 5. If we only wanted to retrieve data from the __males__ and __females__ columns where that criteria were true, we'd need to chain additional operations to the Spark DataFrame. To return the results in descending order instead of ascending order, we'd have to chain another method. The DataFrame methods are quick and powerful for simple queries, but chaining them can be cumbersome for more advanced queries.

SQL shines at expressing complex logic in a more compact manner. Let's brush up on SQL by writing a query that expresses more specific criteria.

#### Instructions
Write and run a SQL query that returns:
* The __males__ and __females__ columns (in that order) where __age__ > 5 and __age__ < 15

In [18]:
sqlCtx.sql('SELECT males, females FROM census2010 \
            WHERE age > 5 AND age < 15').show()

+-------+-------+
|  males|females|
+-------+-------+
|2093905|2007781|
|2097080|2010281|
|2101670|2013771|
|2108014|2018603|
|2114217|2023289|
|2118390|2026352|
|2132030|2037286|
|2159943|2060100|
|2195773|2089651|
+-------+-------+



## Mixing Functionality
Because the results of SQL queries are DataFrame objects, we can combine the best aspects of both DataFrames and SQL to enhance our workflow. For example, we can write a SQL query that quickly returns a subset of our data as a DataFrame.

#### Instructions
* Write a SQL query that returns a DataFrame containing the __males__ and __females__ columns from the census2010 table.
* Use the __describe()__ [method](https://spark.apache.org/docs/1.5.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.describe) to calculate summary statistics for the DataFrame and the __show()__ method to display the results.

In [20]:
sqlCtx.sql('SELECT males, females FROM census2010').describe().show()

+-------+------------------+-----------------+
|summary|             males|          females|
+-------+------------------+-----------------+
|  count|               101|              101|
|   mean|1520095.3168316833|1571460.287128713|
| stddev| 818587.2080168233|748671.0493484349|
|    min|              4612|            25673|
|    max|           2285990|          2331572|
+-------+------------------+-----------------+

